In [3]:
import numpy as np
import control
import scipy.io
def normalize_angle(angle):
    while angle > np.pi:
        angle -= 2 * np.pi
    while angle <= -np.pi:
        angle += 2 * np.pi
    return angle

class FeedForwardLQR():
    def __init__(self, A, B, C, D, Q, R):    
        # FeedForward
        A=np.array(A)
        B=np.array(B)
        C=np.array(C)
        D=np.array(D)
        Q=np.array(Q)
        R=np.array(R)
        
        A_ffw = np.vstack((np.hstack((A, B)), np.hstack((C, D))))
        Nx_Nu = np.dot(np.linalg.pinv(A_ffw), np.vstack([np.zeros((A.shape[0], B.shape[1])), np.eye(B.shape[1])]))
        self.Nx = Nx_Nu[:len(A), :]
        self.Nu = Nx_Nu[len(A):, :]
        
        # Full-State Feedback Gain
        self.K, _, _ = control.lqr(A, B, Q, R)

    def compute_u(self, states, desired_states):
        u = self.Nu @ np.array(desired_states) - self.K @ (np.array(states) - self.Nx @ np.array(desired_states))
        theta, phi, thrust = u[0], u[1], u[2]

        return normalize_angle(theta), normalize_angle(phi), thrust

In [5]:
m = 0.035  # mass of the drone in kg
g = 9.81  # gravity
Kp = 1 # Proportional gain
a_max = 12.4
kvx = 0.03 
kvy = 0.03
kvz = 0

# Matrices A, B, C, D
A = np.array([[-kvx/m, 0, 0],
                    [0, -kvy/m, 0],
                    [0, 0, -kvz/m]])

B = np.array([[g, 0, 0],
                    [0, g, 0],
                    [0, 0, 1/m]])

C = np.array([[1, 0, 0],
                    [0, 1, 0],
                    [0, 0, 1]])

D = np.zeros((C.shape[0], B.shape[1]))

# LQR Matricies
Q = np.array([[10, 0, 0],
                    [0, 10, 0],
                    [0, 0, 1]])

R = np.array([[25, 0, 0],
                    [0, 25, 0],
                    [0, 0, 40]])

fflqr = FeedForwardLQR(A, B, C, D, Q, R)

# print Nu and Nx matrices

print("Nu: ", fflqr.Nu)
print()
print("Nx: ", fflqr.Nx)

Nu:  [[0.0873744 0.        0.       ]
 [0.        0.0873744 0.       ]
 [0.        0.        0.       ]]

Nx:  [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
